## Random Forest Classification : sklearn.ensemble

to make decision of whther the given mushroom is edible or poisonous

In [2]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('mushrooms.csv')
data.head()
data = data[data['stalk-root'] != '?']
data = data.drop(columns = ['cap-surface','gill-spacing','gill-attachment','stalk-surface-above-ring','stalk-surface-below-ring','ring-number','ring-type','spore-print-color'],axis = 1)


 Class - Label (p- poisonous,e- edible)

### step 2: Preparing data
feature selection

In [4]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

X=data.iloc[:,1:].values  #features
    

#y = data.iloc[:,0]  #Class label
#y = y.values.reshape(-1,1)

def convert_str_to_int(data,column):
    unique = set([row for row in data[column]])
    lookup = {}
    d= []
    for i,val in enumerate(unique):
        #print(i,val)
        lookup[val] = i
    for row in data[column]:
        row = lookup[row]
        d.append(row)
    return np.array(d)
y= convert_str_to_int(data,'class')


#### one-hot encoding:
Using one-hot encoding , we can change the features from categorical value to the numerical value

### Step 3: dataset splitting (cross_validation)
train_test_split

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3, random_state=42)

In [7]:
#Using one hot encoder.
    
#for training data
from sklearn.preprocessing import OneHotEncoder

encode = OneHotEncoder(sparse = False)
X_train_encode = encode.fit_transform(X_train)
print(X_train_encode.shape)

X_test_encode = encode.fit_transform(X_test)
print(X_test_encode.shape)

(2099, 55)
(900, 55)


### Step 4: Using Random Forest Classifier


In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
classifier = RandomForestClassifier()
classifier

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [10]:
model = classifier.fit(X_train_encode,y_train)
model

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [11]:
y_pred = model.predict(X_test_encode)

In [12]:
#accuracy
from sklearn.metrics import confusion_matrix,r2_score,classification_report,accuracy_score
cm = confusion_matrix(y_test,y_pred)
cm

array([[807,   0],
       [  0,  93]], dtype=int64)

In [13]:
accuracy = classification_report(y_test,y_pred)

Witnessing how consistent the performance is, we can now conclude the model is indeed performing well and not overfitting.

In [14]:
accuracy

'              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00       807\n           1       1.00      1.00      1.00        93\n\n   micro avg       1.00      1.00      1.00       900\n   macro avg       1.00      1.00      1.00       900\nweighted avg       1.00      1.00      1.00       900\n'

One encoding option is to convert the distinct values for each feature into integers (LabelEncoder from Scikit-learn is handy). For example, the values for veil color are brown, orange, white, and yellow, but they can be represented by 1, 2, 3 and 4, respectively. Unfortunately, this is strategy has nonsensical implications— yellow isn't four times the value of brown! Now if our features were ordinal categorical variables, such as T-shirt size (small, medium, large), this strategy could have worked.

Instead, our features are nominal categorical variables with no intrinsic order. Therefore, we'll need to perform one-hot encoding, in which each feature with $z$ possible values is converted into $z$ binary features, only one of which is "on". We could use OneHotEncoder from Scikit-learn to execute this strategy, but this preprocessor requires the categories to be already encoded as integers. We can actually skip this extra step if we instead use get_dummies() from Pandas, which does everything in one go and provides appropriate column labels to boot!

We can now use our model to decide if an Agaricus or Lepiota mushroom is edible.

In [16]:
accuracy = accuracy_score(y_test,y_pred)
accuracy

1.0